Wikipedia Code Referenced from https://github.com/sahanbull/context-agnostic-engagement/blob/0472e76c6bd00d686b235d844e2fb4d71649400c/context_agnostic_engagement/feature_extraction/_api_utils.py#L47

In [ ]:
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Text Files Generator 
import numpy as np
master_dict = np.load('/content/drive/MyDrive/IRDM/files_rel.npy',allow_pickle='TRUE').item()


In [ ]:
# Creation of batches and the text file is read above
# You can split the data in batches and run the wikification below in batch saving 

batch_1k_1=list(master_dict.keys())[130000:131000]
batch_1k_2= list(master_dict.keys())[131000:132000]
batch_1k_3= list(master_dict.keys())[132000:133000]
batch_1k_4= list(master_dict.keys())[133000:134000]
batch_1k_5= list(master_dict.keys())[134000:135000]
batch_1k_6= list(master_dict.keys())[135000:136000]
batch_1k_7= list(master_dict.keys())[136000:137000]
batch_1k_8= list(master_dict.keys())[137000:128000]
batch_1k_9= list(master_dict.keys())[138000:139000]


In [ ]:
len(master_dict)

138605

In [ ]:
len(master_dict)

14295

In [ ]:
np.save('/content/drive/MyDrive/IRDM/files_rel_train.npy', master_dict) 


In [ ]:
from collections import defaultdict
import time

import requests
import ujson as json

ERROR_KEY = u'error'
STATUS_FIELD = u'status'

URL_FIELD = u'url'
PAGERANK_FIELD = u'pageRank'

COSINE_FIELD = u'cosine'

ANNOTATION_DATA_FIELD = u'annotation_data'

_WIKIFIER_URL = u"http://www.wikifier.org/annotate-article"
_WIKIFIER_MAX_SERVER_LIMIT = 25000
WIKIFIER_MAX_CHAR_CEILING = round(_WIKIFIER_MAX_SERVER_LIMIT * .99)  # 99% of max allowed num chars for a post request




import re

SENTENCE_AGGREGATOR = " "
LEN_SENTENCE_AGGR = len(SENTENCE_AGGREGATOR)



import re
import requests

def get_page_tag(url, title_re=re.compile(r'<title>(.*?)</title>', re.UNICODE )):
    
    """
    Retrieves the title tag from a given url (or actually any tag if you want..)
    
    Requirements:
    requests (pip install requests)
    
    Example usage:
    
    Simple
        >>> title = get_page_tag("http://www.tv2.no/nyheter/innenriks/her-ligger-24-tonn-kylling-i-groefta-3692748.html")
    
    Supply your own regular expression to filter out what you want to be returned
    
        >>> myre = re.compile(r'<title>(.*?) - TV 2 Nyhetene</title>', re.UNICODE
        >>> title = get_page_tag("http://www.tv2.no/nyheter/innenriks/her-ligger-24-tonn-kylling-i-groefta-3692748.html", title_re=myre)
        
    """

    r = requests.get(url)
    if r.status_code == 200:
        match = title_re.search(r.text)
        if match:
            return match.group(1)
        return Exception("No match for title in page")
    raise Exception(r.status_code)


def _make_regex_with_escapes(escapers):
    words_regex = r'{}[^_\W]+{}'

    temp_regexes = []
    for escaper_pair in escapers:
        (start, end) = escaper_pair
        temp_regexes.append(words_regex.format(start, end))

    return r"|".join(temp_regexes)


def shallow_word_segment(phrase, escape_pairs=None):
    """ Takes in a string phrase and segments it into words based on simple regex
    Args:
        phrase (str): phrase to be segmented to words
        escape_pairs ([(str, str)]): list of tuples where each tuple is a pair of substrings that should not be
                    used as word seperators. The motivation is to escapte special tags such as [HESITATION], ~SILENCE~
                    IMPORTANT: Row regex has to be used when definng escapte pairs
                        ("[", "]") will not work as [] are special chars in regex. Instead ("\[", "\]")
    Returns:
        ([str]): list of words extracted from the phrase
    """
    if escape_pairs is None:
        escape_pairs = []

    escape_pairs.append(("", ""))

    _regex = _make_regex_with_escapes(escape_pairs)
    return re.findall(_regex, phrase, flags=re.UNICODE)


def _segment_sentences(text):
    """segments a text into a set of sentences
    Args:
        text:
    Returns:
    """
    import en_core_web_sm
    nlp = en_core_web_sm.load()

    text_sentences = nlp(text)

    for sentence in text_sentences.sents:
        yield sentence.text


def partition_text(text, max_size):
    """takes a text string and creates a list of substrings that are shorter than a given length
    Args:
        text (str): text to be partitioned (usually a lecture transcript)
        max_size (int): maximum number of characters one partition should contain
    Returns:
        chunks([str]): list of sub strings where each substring is shorter than the given length
    """
    # get sentences
    sentences = _segment_sentences(text)

    chunks = []

    temp_sents = []
    temp_len = 0
    for sentence in sentences:
        len_sentence = len(sentence)
        expected_len = temp_len + LEN_SENTENCE_AGGR + len_sentence  # estimate length cost
        if expected_len > max_size:  # if it goes above threshold,
            if len(temp_sents) > 0:
                chunks.append(SENTENCE_AGGREGATOR.join(temp_sents))  # first load the preceding chunk
                temp_sents = []
                temp_len = 0

        temp_sents.append(sentence)  # then aggregate the sentence to the temp chunk
        temp_len += len_sentence

    if len(temp_sents) > 0:
        chunks.append(SENTENCE_AGGREGATOR.join(temp_sents))  # send the remainder chunk

    return chunks






def _get_wikififier_concepts(resp):
    annotations = [{URL_FIELD: ann[URL_FIELD],
                    COSINE_FIELD: ann[COSINE_FIELD],
                    PAGERANK_FIELD: ann[PAGERANK_FIELD]} for ann in resp.get("annotations", [])]

    return {
        ANNOTATION_DATA_FIELD: annotations,
        STATUS_FIELD: resp[STATUS_FIELD]
    }


def _get_wikifier_response(text, api_key, df_ignore, words_ignore):
    params = {"text": text,
              "userKey": api_key,
              "nTopDfValuesToIgnore": df_ignore,
              "nWordsToIgnoreFromList": words_ignore}
    r = requests.post(_WIKIFIER_URL, params)
    if r.status_code == 200:
        resp = json.loads(r.content)
        if ERROR_KEY in resp:
            raise ValueError("error in response : {}".format(resp[ERROR_KEY]))
        return resp
    else:
        raise ValueError("http status code 200 expected, got status code {} instead".format(r.status_code))


def wikify(text, key, df_ignore, words_ignore):
    """This function takes in a text representation of a lecture transcript and associates relevant Wikipedia topics to
    it using www.wikifier.org entity linking technology.
    Args:
        text (str): text that needs to be Wikified (usually lecture transcript string)
        key (str): API key for Wikifier obtained from http://www.wikifier.org/register.html
        df_ignore (int): Most common words to ignore based on Document frequency
        words_ignore (int): Most common words to ignore based on Term frequency
    Returns:
        [{key:val}]: a dict with status of the request and the list of Wiki topics linked to the text
    """
    try:
        resp = _get_wikifier_response(text, key, df_ignore, words_ignore)
        resp[STATUS_FIELD] = 'success'
    except ValueError as e:
        try:
            STATUS_ = e.message
        except:
            STATUS_ = e.args[0]
        return {
            STATUS_FIELD: STATUS_
        }
    time.sleep(0.5)
    return _get_wikififier_concepts(resp)
# values for Doc Frequency and Words to Ignore, more details about these variables
# found at: http://www.wikifier.org/info.html
DF_IGNORE_VAL = 50
WORDS_IGNORE_VAL = 50


def get_wikipedia_topic_features(text, api_key, chunk_size=5000):
    """ get Wikification for the transcript using http://www.wikifier.org
    Args:
        text (str): text that needs to be Wikified
        api_key (str): API key for Wikifier obtained from http://www.wikifier.org/register.html
        chunk_size (int): maximum number of characters that need included in each Wikified fragment.
    Returns:
        enrichments ([{str: val}]): list of annotated chunks from the transcript
    """
    text_partitions = partition_text(text, max_size=chunk_size)

    enrichments = []
    i = 1
    for text_part in text_partitions:
        temp_record = {}
        annotations = wikify(text_part, api_key, DF_IGNORE_VAL, WORDS_IGNORE_VAL)
        temp_record["part"] = i
        temp_record["text"] = text_part
        temp_record["annotations"] = annotations
        enrichments.append(temp_record)
        i += 1

    return enrichments


def get_ranked_topics(chunks, option, top_n):
    """ ranks the topics using the aggregated score across multiple Wikified chunks of the text.
    Args:
        chunks ([{str: val}]): list of Wikified chunks for the transcript
        option {str}: pageRank or cosine
        top_n (int): n top ranked topics of interest
    Returns:
        final_rec ({str:val}): dict with key for top_n_url or top_n_value and the URL or value of the topic
    """
    chunks = list(chunks)

    total_length = sum([len(part["text"]) for part in chunks])

    records = defaultdict(list)
    for part in chunks:
        annotations = part["annotations"]["annotation_data"]
        weight = len(part["text"])
        norm = weight / total_length
        for concept in annotations:
            url = concept["url"]
            val = concept.get(option, 0.)
            records[url].append(val * norm)

    rec = [(title, sum(val)) for title, val in records.items()]

    # sort by normalised weight
    rec.sort(key=lambda l: l[1], reverse=True)
    n_recs = rec[:top_n]

    final_rec = {}
    for idx, item in enumerate(n_recs):
        url, val = item
        _idx = idx + 1
        final_rec["topic_{}_{}_url".format(_idx, option)] = url
        final_rec["topic_{}_{}_val".format(_idx, option)] = val

    return final_rec


def get_authority_wiki_features(text, api_key, top_n):
    """ returns top-n most authoritative Wikipedia topics with PageRank scores.
    Calculated using http://www.wikifier.org/
    Args:
        text (str): text that needs to be Wikified for authority
        api_key (str): API key for Wikifier obtained from http://www.wikifier.org/register.html
        top_n (int): n top ranking topics to be returned with PageRank scores
    Returns:
        ranked_topic_records ({str:val}): dict with key for top_n_url or top_n_value and the URL or value of the topic
    """
    enriched_chunks = get_wikipedia_topic_features(text, api_key)
    ranked_topic_records = get_ranked_topics(enriched_chunks, "pageRank", top_n)

    return ranked_topic_records


def get_coverage_wiki_features(text, api_key, top_n):
    """ returns top-n most covered Wikipedia topics with cosine similarity scores.
    Calculated using http://www.wikifier.org/
    Args:
        text (str): text that needs to be Wikified for coverage
        api_key (str): API key for Wikifier obtained from http://www.wikifier.org/register.html
        top_n (int): n top ranking topics to be returned with cosine scores
    Returns:
        ranked_topic_records ({str:val}): dict with key for top_n_url or top_n_value and the URL or value of the topic
    """
    enriched_chunks = get_wikipedia_topic_features(text, api_key)
    ranked_topic_records = get_ranked_topics(enriched_chunks, "cosine", top_n)

    return ranked_topic_records



In [ ]:
import requests
import spacy
from collections import Counter
import matplotlib.pyplot as plt
import spacy

import argparse
import sys
import string
import spacy
import xml.etree.ElementTree as ET


def convert_string(hj):
  new_s=[]
  for token in hj:
      new_s.append(str(token).strip(string.punctuation))
  return ''.join(new_s)

def get_concept_list(response):
  concepts=[]
  for i in range(len(response["annotation_data"])):
    url = response["annotation_data"][i]['url']
    cos = response["annotation_data"][i]['cosine']
    pr  = response["annotation_data"][i]['pageRank']


    concepts.append([convert_string(' '.join(url.split('/')[-1].split('_'))),cos,pr])
  return concepts

In [ ]:
tab_key= """qgllexjwcwghzerkvodpdawsfsiexg"""

In [ ]:
sample_input= """Set the crib man smoking at Asian fantasy dragging them teacakes thing about how beautiful life is man. Check it out smoke em, if you got em came to the top all the way from the bottom and I'm just trying to count my blessings. I wouldn't leave you with. All right friends, very smooth intro right there. Shout out to fir sellout the fern. I'm the mic Mike's weeks with no episode 16 apologies already blowing it very special podcast episode or bad depending on if you like the kid or not. We got our buddy. That's a debate into at boom Shenanigans, right? There's there dot between that seems underscore right? That's right. That's what everyone. Yeah. We got her buddy Sammy. He's an actor comedian and most importantly if you fall. Me on Instagram our CEO of our fantasy football team. That's your value to me at least a me on my team by the way. Yeah. I think he's rolling you guys I think so. Yeah, I could roll. Yes. It's going on nice going on man what they know and he stopped and had a show in Encinitas. He has a show this Saturday in Clermont at the comedy Palace nice. He has another one in Vista on Friday, October 25th coming up. So couple shows you here's my biggest group either. I give you disclose that the more that I know that I forgot was like, you know always 8 minutes 10 minutes and"""

In [ ]:
for i in  range(10):
  get_response=wikify(sample_input,tab_key,50,50)
  converted_concepts=get_concept_list(get_response)

In [ ]:
def shadowfax(dict_with_files,mapper):
  doc_concept={}
  for i in range(len(mapper)):
    key=mapper[i]
    doc_concept.update({key:get_concept_list(wikify(dict_with_files[key],tab_key,50,50))})
  return doc_concept


batch_one=shadowfax(master_dict,batch_1k_1)
np.save('/content/drive/MyDrive/IRDM/batch_tk_131.npy', batch_one) 

In [ ]:
def shadowfax(dict_with_files,mapper):
  doc_concept={}
  for i in range(len(mapper)):
    key=mapper[i]
    doc_concept.update({key:get_concept_list(wikify(dict_with_files[key],tab_key,50,50))})
  return doc_concept


batch_one=shadowfax(master_dict,batch_1k_2)
np.save('/content/drive/MyDrive/IRDM/batch_tk_132.npy', batch_one) 

In [ ]:
def shadowfax(dict_with_files,mapper):
  doc_concept={}
  for i in range(len(mapper)):
    key=mapper[i]
    doc_concept.update({key:get_concept_list(wikify(dict_with_files[key],tab_key,50,50))})
  return doc_concept


batch_one=shadowfax(master_dict,batch_1k_3)
np.save('/content/drive/MyDrive/IRDM/batch_tk_133.npy', batch_one) 

In [ ]:
def shadowfax(dict_with_files,mapper):
  doc_concept={}
  for i in range(len(mapper)):
    key=mapper[i]
    doc_concept.update({key:get_concept_list(wikify(dict_with_files[key],tab_key,50,50))})
  return doc_concept


batch_one=shadowfax(master_dict,batch_1k_4)
np.save('/content/drive/MyDrive/IRDM/batch_tk_314.npy', batch_one) 

In [ ]:
def shadowfax(dict_with_files,mapper):
  doc_concept={}
  for i in range(len(mapper)):
    key=mapper[i]
    doc_concept.update({key:get_concept_list(wikify(dict_with_files[key],tab_key,50,50))})
  return doc_concept


batch_one=shadowfax(master_dict,batch_1k_5)
np.save('/content/drive/MyDrive/IRDM/batch_tk_135.npy', batch_one) 

In [ ]:
def shadowfax(dict_with_files,mapper):
  doc_concept={}
  for i in range(len(mapper)):
    key=mapper[i]
    doc_concept.update({key:get_concept_list(wikify(dict_with_files[key],tab_key,50,50))})
  return doc_concept


batch_one=shadowfax(master_dict,batch_1k_6)
np.save('/content/drive/MyDrive/IRDM/batch_tk_136.npy', batch_one) 

In [ ]:
def shadowfax(dict_with_files,mapper):
  doc_concept={}
  for i in range(len(mapper)):
    key=mapper[i]
    doc_concept.update({key:get_concept_list(wikify(dict_with_files[key],tab_key,50,50))})
  return doc_concept


batch_one=shadowfax(master_dict,batch_1k_7)
np.save('/content/drive/MyDrive/IRDM/batch_tk_137.npy', batch_one) 

In [ ]:
def shadowfax(dict_with_files,mapper):
  doc_concept={}
  for i in range(len(mapper)):
    key=mapper[i]
    doc_concept.update({key:get_concept_list(wikify(dict_with_files[key],tab_key,50,50))})
  return doc_concept


batch_one=shadowfax(master_dict,batch_1k_8)
np.save('/content/drive/MyDrive/IRDM/batch_tk_138.npy', batch_one) 

In [ ]:
def shadowfax(dict_with_files,mapper):
  doc_concept={}
  for i in range(len(mapper)):
    key=mapper[i]
    doc_concept.update({key:get_concept_list(wikify(dict_with_files[key],tab_key,50,50))})
  return doc_concept


batch_one=shadowfax(master_dict,batch_1k_9)
np.save('/content/drive/MyDrive/IRDM/batch_tk_139.npy', batch_one) 

In [ ]:
def shadowfax(dict_with_files,mapper):
  doc_concept={}
  for i in range(len(mapper)):
    key=mapper[i]
    doc_concept.update({key:get_concept_list(wikify(dict_with_files[key],tab_key,50,50))})
  return doc_concept


batch_one=shadowfax(master_dict,batch_1k_10)
np.save('/content/drive/MyDrive/IRDM/batch_tk_120.npy', batch_one) 